# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)
league_data = pd.read_sql("SELECT * FROM League;", db)
match_data = pd.read_sql("SELECT * FROM Match;", db)
player_attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)

In [4]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [5]:
players_180_190 = player_data[(player_data["height"] >= 180) &
                              (player_data["height"] <= 190)].shape[0]

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [6]:
player_data["birthday"] = pd.to_datetime(player_data["birthday"])
players_1980 = player_data[player_data["birthday"].dt.year == 1980].shape[0]

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [7]:
x = player_data.sort_values(by = ["weight", "player_name"], ascending = [False, True])["player_name"][0:10]
highest_players = x.to_list()

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [8]:
x = pd.DataFrame(player_data[(player_data["birthday"].dt.year >= 1980) & (player_data["birthday"].dt.year <= 1990)])
x["year"] = player_data["birthday"].dt.year
x["id"] = 1
grouped = x["id"].groupby(x["year"]).sum()
years_born_players = list(pd.DataFrame({'y': grouped.index, 's': grouped.values}).to_records(index=False))
years_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [9]:
x = player_data
x["name"] = x["player_name"].str[0:7]
x = x[x["name"] == "Adriano"]
adriano_mean, adriano_std = x["height"].mean(), x["height"].std()
adriano_mean, adriano_std

(182.11800000000002, 5.361479480723788)

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [10]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
x = player_data

x["day"] = x["birthday"].dt.dayofweek
grouped = x["id"].groupby(x["day"]).sum()

dow_with_min_players_born = days[grouped.idxmin()]

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [11]:
m = match_data
m["ident"] = 1
l = league_data
grouped_by_leagues = m["ident"].groupby(m["league_id"]).sum()
l["total_matches"] = grouped_by_leagues.values
l = l.sort_values(by = ["total_matches", "name"], ascending = [False, True])
league_most_matches = l[0:1]["name"].to_list()[0]
league_most_matches

'England Premier League'

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [12]:
s = pd.concat([match_data["home_player_1"], match_data["home_player_2"]])
s = pd.concat([s, match_data["home_player_3"]])
s = pd.concat([s, match_data["home_player_4"]])
s = pd.concat([s, match_data["home_player_5"]])
s = pd.concat([s, match_data["home_player_6"]])
s = pd.concat([s, match_data["home_player_7"]])
s = pd.concat([s, match_data["home_player_8"]])
s = pd.concat([s, match_data["home_player_9"]])
s = pd.concat([s, match_data["home_player_10"]])
s = pd.concat([s, match_data["home_player_11"]])

s = pd.concat([s, match_data["away_player_1"]])
s = pd.concat([s, match_data["away_player_2"]])
s = pd.concat([s, match_data["away_player_3"]])
s = pd.concat([s, match_data["away_player_4"]])
s = pd.concat([s, match_data["away_player_5"]])
s = pd.concat([s, match_data["away_player_6"]])
s = pd.concat([s, match_data["away_player_7"]])
s = pd.concat([s, match_data["away_player_8"]])
s = pd.concat([s, match_data["away_player_9"]])
s = pd.concat([s, match_data["away_player_10"]])
s = pd.concat([s, match_data["away_player_11"]])

df = pd.DataFrame(s)
df["ident"] = 1
df
grouped = df["ident"].groupby(df[0]).sum().sort_values()
# grouped#.index[0]
#player_data[player_data["player_api_id"] == grouped.index[0]]["player_name"].to_list()[0]
id = grouped.idxmax()
id
max_matches_player = player_data[player_data["player_api_id"] == id]["player_name"].to_list()[0]
max_matches_player

'Steve Mandanda'

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [13]:
x = player_attributes_data
x = x.drop(columns = [ "id", "player_fifa_api_id", "player_api_id", "date", "preferred_foot", "attacking_work_rate", "defensive_work_rate"])
abs_pearson = x.corr(method="pearson").abs()
s = abs_pearson.unstack()
s = s[s!=1]
so = s.sort_values(kind="quicksort", ascending=False)
top_five = so[0:10:2]
top_correlated_features = top_five.index.to_list()
top_correlated_features

[('gk_reflexes', 'gk_positioning'),
 ('gk_reflexes', 'gk_handling'),
 ('gk_positioning', 'gk_handling'),
 ('standing_tackle', 'sliding_tackle'),
 ('marking', 'standing_tackle')]

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [14]:
import scipy

from scipy.spatial import distance

x = player_attributes_data#.drop(columns=["ident"])

#print(player_data[player_data["name"] == "Neymar"])

drop_list = ["id", "player_fifa_api_id", "player_api_id", "date", "preferred_foot", "attacking_work_rate", "defensive_work_rate"]

x.groupby(['player_api_id'], sort=False)['date'].max()
idx = x.groupby(['player_api_id'])['date'].transform(max) == x['date']
x = x[idx]

y = player_data
y["name"] = y["player_name"].str[0:6]
neymar_id = y[y["name"] == "Neymar"]["player_api_id"].to_list()[0]

#print(neymar_id)

neymar = x[x["player_api_id"] == neymar_id]

#print(neymar)



x = x.drop(columns = drop_list)
neymar = neymar.drop(columns = drop_list)

#x.loc[[131464]]
#distance.cdist(x.loc[[131464]], neymar, 'euclidean')


#neymar

distances = distance.cdist(neymar.to_numpy(), x.to_numpy(), 'euclidean') # x.to_numpy()
# distances[0][0:1000]
# x.index
series = pd.Series(data = distances[0], index = x.index)
#series = series.sort_values()
series

x["dist"] = series
x = x.sort_values(by = "dist")
ind = x.index[1:6] # 131454, 140773, 47247, 64607
ids = player_attributes_data.iloc[ind]["player_api_id"]#.to_list()
#print(ids)
result = player_data.loc[player_data["player_api_id"].isin(ids)]
#print(result)
df = pd.DataFrame(ids)
df=df.merge(result, how='left', on='player_api_id')


neymar_similarities = df["player_name"].to_list()
neymar_similarities

['Paulo Dybala',
 'Eden Hazard',
 'Giovani dos Santos',
 'Arjen Robben',
 'Lionel Messi']

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [15]:
league_data
id_german_bundesliga = league_data[league_data["name"] == "Germany 1. Bundesliga"]["id"].to_list()[0]
id_german_bundesliga

id_bor = team_data[team_data["team_long_name"] == "Borussia Dortmund"]
id_bor = id_bor["team_api_id"].to_list()[0]
id_bor

borussia_bundesliga_2008_2009_matches = match_data[(match_data["league_id"] == id_german_bundesliga) & (match_data["season"] == "2008/2009") & (match_data["home_team_api_id"] == id_bor)].shape[0]
borussia_bundesliga_2008_2009_matches

17

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [16]:
league_data
id_german_bundesliga = league_data[league_data["name"] == "Germany 1. Bundesliga"]["id"].to_list()[0]
id_german_bundesliga

years = "2008/2009"

df = match_data[(match_data["league_id"] == id_german_bundesliga) & (match_data["season"] == years)]

s = pd.concat([df["home_team_api_id"], df["away_team_api_id"]])

team_most_matches_bundesliga_2008_2009 = s.value_counts().max()
team_most_matches_bundesliga_2008_2009

34

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [17]:
id_ars = team_data[team_data["team_long_name"] == "Arsenal"]
id_ars = id_ars["team_api_id"].to_list()[0]
id_ars

df = match_data[ (match_data["home_team_api_id"] == id_ars) & (match_data["season"] == "2015/2016")]
df = df[df["home_team_goal"] - df["away_team_goal"] > 0]

x = df.shape[0]
x

df = match_data[ (match_data["away_team_api_id"] == id_ars) & (match_data["season"] == "2015/2016")]
df = df[df["home_team_goal"] - df["away_team_goal"] < 0]

y = df.shape[0]
y

arsenal_won_matches_2015_2016 = x + y
arsenal_won_matches_2015_2016

20

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [18]:
df = match_data[match_data["season"] == "2015/2016"]

s = pd.concat([df["home_team_api_id"], df["away_team_api_id"]])
s

teams = pd.DataFrame(s)

teams["ident"] = 1


grouped = teams["ident"].groupby(teams[0]).sum()
grouped

grouped_df = pd.DataFrame({"teams_id": grouped.index, "total": grouped.values})
grouped_df # total games

l = df.shape[0]

df = pd.DataFrame(df)

df["winner"] = -1
df.loc[df["home_team_goal"] < df["away_team_goal"], "winner"] = df["away_team_api_id"]
df.loc[df["home_team_goal"] > df["away_team_goal"], "winner"] = df["home_team_api_id"]

df["indent"] = 1
grouped = df["ident"].groupby(df["winner"]).sum()
grouped

grouped_winner = pd.DataFrame({"teams_id": grouped.index, "winner": grouped.values})
grouped_winner = grouped_winner.drop([0]) #=  grouped_winner.drop(0)
grouped_winner

grouped_df.index = grouped_df.index + 1
grouped_df
grouped_winner["winner"] = grouped_winner["winner"]/grouped_df["total"]
grouped_winner = grouped_winner.sort_values(by = "winner", ascending= False)
grouped_winner
#s = grouped_winner["winner"]/grouped_df["total"]

#s.sort_values()

#print(grouped_df)
#print(grouped_winner)
team_id = grouped_winner.iloc[0].to_list()[0]
team_highest_winrate_2015_2016 = team_data[team_data["team_api_id"] == team_id]["team_long_name"].to_list()[0]
team_highest_winrate_2015_2016

'SL Benfica'

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [63]:
league_id =  league_data[league_data["name"] == "England Premier League"]["id"].to_list()[0]

df = pd.DataFrame(match_data[(match_data["league_id"] == league_id) & (match_data["season"] == "2010/2011")])
df["date"] = pd.to_datetime(df["date"])
df

part1 = df[["date", "home_team_api_id"]]
part1
part1 = part1.rename(columns = {"home_team_api_id": "id"})
part1
part2 = df[["date", "away_team_api_id"]]
part2 = part2.rename(columns = {"away_team_api_id": "id"})
part2

df = pd.concat([part1, part2], ignore_index=True)

df = df.sort_values(by="date")

######### df.groupby("id")["date"].diff().max().days

highest_gap_england_2010_2011 = df.groupby("id").apply(lambda x: x["date"].diff().max()).max().days

########### highest_gap_england_2010_2011 = 223

highest_gap_england_2010_2011

18

### Warning! Do not change anything in the area below

In [ ]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")